# Yelp API Restaurant Calls By City and Top Ratings

- Similar to city_calls code but sorts by top rated restaurants 
- Effort to target and avoid skipping desired data (due to 1k query limit per search)
- See city_calls.ipynb for more detailed explanation on api call function

#   

In [2]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import csv

# Import API key
from api_keys import api_key

### Perform API calls

- Dataframe from cities.csv
- Build function that calls and writes restaurant data to new csv, given a city
- Run list of cities through function

In [3]:
# Create dataframe from csv file
cities_df = pd.read_csv("cities.csv", names=["City", "County", "Population", "Area (sq. mi)"])
cities_df

,City,County,Population,Area (sq. mi)
0,Alameda,Alameda,"73,812",10.61
1,Albany,Alameda,"18,539",1.79
2,American Canyon,Napa,"19,454",4.84
3,Antioch,Contra Costa,"102,372",28.35
4,Atherton,San Mateo,"6,914",5.02
5,Belmont,San Mateo,"25,835",4.62
6,Belvedere,Marin,"2,068",0.52
7,Benicia,Solano,"26,997",12.93
8,Berkeley,Alameda,"112,580",10.47
9,Brentwood,Contra Costa,"51,481",14.79


In [8]:
# Function returns up to 1000 restaurant listings for input city sorted by Yelp-adjusted best rating
def get_restaurants_rating(city, api_key):
    
    url = "https://api.yelp.com/v3/businesses/search"
    headers = {"Authorization": "Bearer %s" % api_key}
    restaurant_data = []
    count = 0
    
    # Increases search return limit
    for offset in range(0, 1000, 50):
        
        # Set parameters and pass into API calls
        params = {"term": "restaurants", "sort_by": "rating", "location":city + ", CA", "limit":50, 
                  "offset":offset}
        req = requests.get(url, params=params, headers=headers)
        
        # Breaks if error occurs with search
        if req.status_code == 400:
            break
        elif req.status_code == 200:
            
            # Convert to json
            response = req.json()
            
            # Log history
            count += 1
            print(f"Now processing query set {count} of max 20 for {city}")
            
            # Breaks if no further entries in query
            if response["businesses"] == []:
                break
            
            else:
                # Iterate through business results and extract data
                for biz in response["businesses"]:
                    
                    # Logic to replace missing price level data with NaN
                    if "price" not in biz:
                        restaurant_data.append([city, biz["name"], biz["coordinates"]["latitude"], 
                                                biz["coordinates"]["longitude"], biz["location"]["address1"], 
                                                biz["location"]["zip_code"], biz["rating"], 
                                                biz["review_count"], "", biz["categories"][0]["title"], 
                                                biz["id"]])

                    # Replace missing category data with empty string
                    elif biz["categories"] == []:
                        restaurant_data.append([city, biz["name"], biz["coordinates"]["latitude"], 
                                                biz["coordinates"]["longitude"], biz["location"]["address1"], 
                                                biz["location"]["zip_code"], biz["rating"], 
                                                biz["review_count"], biz["price"], "", biz["id"]])

                    else:
                        restaurant_data.append([city, biz["name"], biz["coordinates"]["latitude"], 
                                                biz["coordinates"]["longitude"], biz["location"]["address1"], 
                                                biz["location"]["zip_code"], biz["rating"], 
                                                biz["review_count"], biz["price"], 
                                                biz["categories"][0]["title"], biz["id"]])

    # Write to csv
    with open('restaurant_data_rating.csv', 'a', encoding="utf-8") as csvFile:
        writer = csv.writer(csvFile)
        writer.writerows(restaurant_data)
    csvFile.close()
    
    # Returns total count of restaurants in city
    return response["total"]


In [9]:
# Track # of cities processed
count = 0

# List to track total restaurants found in city
totals_count = []

print("LOG HISTORY OF API CALLS:")
print("---------------------------")

# Loop thru list of cities in cities_df
for city in cities_df["City"]:
    
    #  Call get_restaurants fn and append to total_count list
    totals_count.append(get_restaurants_rating(city, api_key))
    
    # Print log history
    count += 1
    rem = len(cities_df["City"]) - count
    print("-----------------------------------------")
    if city == cities_df.iloc[-1,0]:
        print("Full list of cities processed!")
    elif city == cities_df.iloc[-2,0]:
        print("Now getting results for final city. Almost there!")
        print("-----------------------------------------")
    else:
        print(f"Data retrieval for {city} complete")
        print(f"Getting results for next city.. there are {rem} cities left")
        print("-----------------------------------------")

cities_df["Total # of Restaurants"] = totals_count

LOG HISTORY OF API CALLS:
---------------------------
Now processing query set 1 of max 20 for Alameda
Now processing query set 2 of max 20 for Alameda
Now processing query set 3 of max 20 for Alameda
Now processing query set 4 of max 20 for Alameda
Now processing query set 5 of max 20 for Alameda
Now processing query set 6 of max 20 for Alameda
Now processing query set 7 of max 20 for Alameda
Now processing query set 8 of max 20 for Alameda
Now processing query set 9 of max 20 for Alameda
Now processing query set 10 of max 20 for Alameda
Now processing query set 11 of max 20 for Alameda
Now processing query set 12 of max 20 for Alameda
Now processing query set 13 of max 20 for Alameda
Now processing query set 14 of max 20 for Alameda
Now processing query set 15 of max 20 for Alameda
Now processing query set 16 of max 20 for Alameda
Now processing query set 17 of max 20 for Alameda
Now processing query set 18 of max 20 for Alameda
Now processing query set 19 of max 20 for Alameda
-----

Now processing query set 1 of max 20 for Colma
Now processing query set 2 of max 20 for Colma
Now processing query set 3 of max 20 for Colma
-----------------------------------------
Data retrieval for Colma complete
Getting results for next city.. there are 84 cities left
-----------------------------------------
Now processing query set 1 of max 20 for Concord
Now processing query set 2 of max 20 for Concord
Now processing query set 3 of max 20 for Concord
Now processing query set 4 of max 20 for Concord
Now processing query set 5 of max 20 for Concord
Now processing query set 6 of max 20 for Concord
Now processing query set 7 of max 20 for Concord
Now processing query set 8 of max 20 for Concord
Now processing query set 9 of max 20 for Concord
Now processing query set 10 of max 20 for Concord
Now processing query set 11 of max 20 for Concord
Now processing query set 12 of max 20 for Concord
Now processing query set 13 of max 20 for Concord
Now processing query set 14 of max 20 for C

Now processing query set 12 of max 20 for Fremont
Now processing query set 13 of max 20 for Fremont
Now processing query set 14 of max 20 for Fremont
Now processing query set 15 of max 20 for Fremont
Now processing query set 16 of max 20 for Fremont
Now processing query set 17 of max 20 for Fremont
Now processing query set 18 of max 20 for Fremont
-----------------------------------------
Data retrieval for Fremont complete
Getting results for next city.. there are 69 cities left
-----------------------------------------
Now processing query set 1 of max 20 for Gilroy
Now processing query set 2 of max 20 for Gilroy
Now processing query set 3 of max 20 for Gilroy
Now processing query set 4 of max 20 for Gilroy
Now processing query set 5 of max 20 for Gilroy
-----------------------------------------
Data retrieval for Gilroy complete
Getting results for next city.. there are 68 cities left
-----------------------------------------
Now processing query set 1 of max 20 for Half Moon Bay
No

Now processing query set 8 of max 20 for Menlo Park
Now processing query set 9 of max 20 for Menlo Park
Now processing query set 10 of max 20 for Menlo Park
Now processing query set 11 of max 20 for Menlo Park
Now processing query set 12 of max 20 for Menlo Park
Now processing query set 13 of max 20 for Menlo Park
-----------------------------------------
Data retrieval for Menlo Park complete
Getting results for next city.. there are 55 cities left
-----------------------------------------
Now processing query set 1 of max 20 for Mill Valley
Now processing query set 2 of max 20 for Mill Valley
Now processing query set 3 of max 20 for Mill Valley
Now processing query set 4 of max 20 for Mill Valley
Now processing query set 5 of max 20 for Mill Valley
-----------------------------------------
Data retrieval for Mill Valley complete
Getting results for next city.. there are 54 cities left
-----------------------------------------
Now processing query set 1 of max 20 for Millbrae
Now proc

Now processing query set 8 of max 20 for Pacifica
Now processing query set 9 of max 20 for Pacifica
Now processing query set 10 of max 20 for Pacifica
Now processing query set 11 of max 20 for Pacifica
Now processing query set 12 of max 20 for Pacifica
Now processing query set 13 of max 20 for Pacifica
Now processing query set 14 of max 20 for Pacifica
Now processing query set 15 of max 20 for Pacifica
Now processing query set 16 of max 20 for Pacifica
-----------------------------------------
Data retrieval for Pacifica complete
Getting results for next city.. there are 41 cities left
-----------------------------------------
Now processing query set 1 of max 20 for Palo Alto
Now processing query set 2 of max 20 for Palo Alto
Now processing query set 3 of max 20 for Palo Alto
Now processing query set 4 of max 20 for Palo Alto
Now processing query set 5 of max 20 for Palo Alto
Now processing query set 6 of max 20 for Palo Alto
Now processing query set 7 of max 20 for Palo Alto
Now proc

Now processing query set 1 of max 20 for Ross
Now processing query set 2 of max 20 for Ross
-----------------------------------------
Data retrieval for Ross complete
Getting results for next city.. there are 28 cities left
-----------------------------------------
Now processing query set 1 of max 20 for St. Helena
Now processing query set 2 of max 20 for St. Helena
-----------------------------------------
Data retrieval for St. Helena complete
Getting results for next city.. there are 27 cities left
-----------------------------------------
Now processing query set 1 of max 20 for San Anselmo
Now processing query set 2 of max 20 for San Anselmo
-----------------------------------------
Data retrieval for San Anselmo complete
Getting results for next city.. there are 26 cities left
-----------------------------------------
Now processing query set 1 of max 20 for San Bruno
Now processing query set 2 of max 20 for San Bruno
Now processing query set 3 of max 20 for San Bruno
Now proces

Now processing query set 1 of max 20 for Santa Rosa
Now processing query set 2 of max 20 for Santa Rosa
Now processing query set 3 of max 20 for Santa Rosa
Now processing query set 4 of max 20 for Santa Rosa
Now processing query set 5 of max 20 for Santa Rosa
Now processing query set 6 of max 20 for Santa Rosa
Now processing query set 7 of max 20 for Santa Rosa
Now processing query set 8 of max 20 for Santa Rosa
Now processing query set 9 of max 20 for Santa Rosa
Now processing query set 10 of max 20 for Santa Rosa
-----------------------------------------
Data retrieval for Santa Rosa complete
Getting results for next city.. there are 15 cities left
-----------------------------------------
Now processing query set 1 of max 20 for Saratoga
Now processing query set 2 of max 20 for Saratoga
Now processing query set 3 of max 20 for Saratoga
Now processing query set 4 of max 20 for Saratoga
Now processing query set 5 of max 20 for Saratoga
Now processing query set 6 of max 20 for Saratoga

Now processing query set 10 of max 20 for Walnut Creek
Now processing query set 11 of max 20 for Walnut Creek
Now processing query set 12 of max 20 for Walnut Creek
Now processing query set 13 of max 20 for Walnut Creek
Now processing query set 14 of max 20 for Walnut Creek
Now processing query set 15 of max 20 for Walnut Creek
Now processing query set 16 of max 20 for Walnut Creek
Now processing query set 17 of max 20 for Walnut Creek
-----------------------------------------
Data retrieval for Walnut Creek complete
Getting results for next city.. there are 3 cities left
-----------------------------------------
Now processing query set 1 of max 20 for Windsor
Now processing query set 2 of max 20 for Windsor
Now processing query set 3 of max 20 for Windsor
-----------------------------------------
Data retrieval for Windsor complete
Getting results for next city.. there are 2 cities left
-----------------------------------------
Now processing query set 1 of max 20 for Woodside
Now pr

In [10]:
# Show cities_df with updated total # of restaurants
cities_df.head()

,City,County,Population,Area (sq. mi),Total # of Restaurants
0,Alameda,Alameda,"73,812",10.61,857
1,Albany,Alameda,"18,539",1.79,128
2,American Canyon,Napa,"19,454",4.84,65
3,Antioch,Contra Costa,"102,372",28.35,383
4,Atherton,San Mateo,"6,914",5.02,178
